In [1]:
from astroquery.gaia import Gaia

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from lightkurve import search_targetpixelfile
import numpy as np
import pandas as pd
from photutilspsf.photutilspsf import read_fits_table

In [3]:
EPIC = 200062544
C = 7
index = 256

tpf = search_targetpixelfile(EPIC).download()

In [4]:
def find_center_and_radius(tpf):
    center = (np.array(tpf.flux[0,:,:].shape)-1)/2
    lo, la = tpf.wcs.wcs_pix2world(*center, 0) #0 indicates we start pixel counting at 0
    lom, lam = tpf.wcs.wcs_pix2world(0, 0, 0)
    radius = np.sqrt((lom - lo)**2 + (lam - la)**2)
    return (lo, la), radius

array([25., 25.])

In [18]:
def get_gaia_astrometry(center, radius):
    coord = SkyCoord(ra=center[0], dec=center[1], unit=(u.degree, u.degree), frame='icrs')
    width = u.Quantity(0.1, u.deg)
    height = u.Quantity(0.1, u.deg)
    r = Gaia.query_object_async(coordinate=coord, radius=radius*u.degree)
    return r.to_pandas()

In [19]:
members = read_fits_table('data/olivares_members_r147.fit')

In [9]:
center, radius = find_center_and_radius(tpf)
others = get_gaia_astrometry(center, radius)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: TableReplaceWarning: converted column 'priam_flags' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: TableReplaceWarning: converted column 'flame_flags' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
others = others[['source_id','ra','dec']]
others = others.rename(index=str, columns={'source_id':'gaia_id'})
others['EPIC'] = 0
others['pmem'] = 0.
ontpf = pd.concat((members,others),ignore_index=True, sort=True)
assert ontpf.shape[0] == members.shape[0] + others.shape[0]